In [1]:
import os
# First we change the directory to the root directory of the project.
os.chdir(os.getcwd().replace("\\notebooks\\retrieve", ""))

In [3]:
from model.models import Agent

In [7]:
# vector search 
param_vs = {
    "embed_name": "bge",
    "param": 
        {
            "search_type": "similarity",
            "search_kwargs": {
                    "k": 5,
                    # "score_threshold": None
            }
        }   
}

# multi query
param_mq = {
    "param": 
        {
            "llm_model": "openai",
            "temperature": 0,
            "top_p": None,
            "top_k": None,
            "frequency_penalty": None,
            "max_tokens": None
        }
}

# keyword extraction - just in case
param_ke = {
    "embed_name": "bge",
    "param":
        {
          
        }
}

# Reranker BGE
param_reranker = {
    "param":
        {
            "k": 5,
        }
}

# LLM Reranker BGE
param_llm_reranker = {
    "param":
        {
            "k": 5,
        }
}

# Response LLM
param_response_llm = {
    "param":{
        
        "llm": {
            "llm_model": "openai",
            "temperature": 0.2,
            "top_p": None,
            "top_k": None,
            "frequency_penalty": None,
            "max_tokens": None
        },
        "shots": {
            "embed_name": "openai",
            "k": 0
        }
    }
}

model_in = {
    "vector_search" : param_vs,
    "multi_query": param_mq,
    # "keyword_extractor": param_ke,
    # "reranker": param_reranker, # Uncomment if you are brave enough and your GPU or CPU is strong enough
    # "llm_reranker": param_llm_reranker, # Uncomment if you are brave enough and your GPU or CPU is strong enough
    "response_llm": param_response_llm 
}

In [8]:
agent = Agent(model_in, DEBUG=True)

Device: cuda:0

Vector Search Initialized
Vector Search Parameters:
Vector Database: Chroma
Embedding Model: HuggingFaceHubEmbeddings
Search Parameters: 
{'k': 5}

Multi Query Initialized
Multi Query Parameters:
LLM Model: 
Verbose: False
Client: <openai.resources.chat.completions.Completions object at 0x000002B29ED76800>
Async_client: <openai.resources.chat.completions.AsyncCompletions object at 0x000002B29ED74730>
Model_name: gpt-3.5-turbo-0125
Temperature: 0.0
Model_kwargs: {'top_p': None, 'frequency_penalty': None}
Openai_api_key: **********
Openai_proxy: 
Max_retries: 2
Streaming: False
N: 1

Prompt: 
Input_variables: ['question']
Input_types: {}
Partial_variables: {}
Template: 
You are an AI language model assistant. Your task is to generate 3 different versions of the given user 
question to retrieve relevant documents from a vector database.
By generating multiple perspectives on the user question,
your goal is to help the user overcome some of the limitations
of distance-based

In [5]:
query = "I have a question regarding the skill class grade. The skill class grade only affects the final grade for the project correct? Also, from what I understand from the grading and assessment, it's worth a total of 10% of the final grade so if you missed one skill class, you would only miss out on 5% of the final grade, correct?"

In [6]:
agent.extract_keywords([query], top_n=5)

[('skill class grade', 0.7712),
 ('skill class', 0.7271),
 ('grading', 0.6889),
 ('final grade', 0.6752),
 ('project', 0.5892)]

In [7]:
# Works
# agent.embed_documents([query])
# agent.embed_query(query)

In [8]:
multi_query_results = agent.generate_multi_query(query) # Works fine, but should be tested in vector space
retrieve_results = agent.retrieve_content(query) # Works fine, but should be evaluated (also we need to try with multi-vector)
# reranker_results = agent.retrieve_content_with_reranker(query, top_n=4)
# llm_reranker_results = agent.retrieve_content_with_llm_reranker(query, top_n=4)
# final_results = agent.apply_reciprocal_rank_fusion([retrieve_results, reranker_results, llm_reranker_results])



Retriever Results:
1. Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.
2. The project grade is a group grade, which applies to all members of the group. The project examiners may deviate (positively or negatively) from the group grade and issue an individual grade for students, if participation and cooperation within a group has not been homogeneous.
3. If a student has participated fully in all but one skill trainings, where fully participating includes that the student has a passing grade for all assignments, the student will receive a grade of 0.5 for the skill classes.
4. The project is graded on three occasions by the examiners. The fi

In [9]:
multi_query_results

{'question': "I have a question regarding the skill class grade. The skill class grade only affects the final grade for the project correct? Also, from what I understand from the grading and assessment, it's worth a total of 10% of the final grade so if you missed one skill class, you would only miss out on 5% of the final grade, correct?",
 'text': ['1. Does the skill class grade solely impact the final project grade, or are there other factors involved?',
  '2. How much weight does the skill class grade carry in the overall final grade calculation, and what happens if a skill class is missed?',
  "3. Can missing a skill class affect the final grade significantly, considering it's worth 10% of the total grade?"]}

In [10]:
reranker_content_results = agent.rerank_content(retrieve_results, query)
llm_reranker_content_results = agent.rerank_content_with_llm(retrieve_results, query)



Reranker content Results:
1. Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.
2. The project is graded on three occasions by the examiners. The first grade is issued after the presentation first phase and accounts for 15% of the final grade. The second grade is issued after the presentation second phase and accounts for 15% of the final grade. The third grade is issued after the final assessment at the end of the third block and accounts for 70% of the grade. These grades are then rescaled in Paragraph 5 to a range 0 to 9.
3. If a student has participated fully in all but one skill trainings, where fully participating includes that the stud

  0%|          | 0/5 [00:00<?, ?it/s]

D:\Code\uni_llm\venv\lib\site-packages\transformers\tokenization_utils_base.py:2707: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(




LLM Reranker content Results:
1. Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.
2. If a student has participated fully in all but one skill trainings, where fully participating includes that the student has a passing grade for all assignments, the student will receive a grade of 0.5 for the skill classes.
3. The project is graded on three occasions by the examiners. The first grade is issued after the presentation first phase and accounts for 15% of the final grade. The second grade is issued after the presentation second phase and accounts for 15% of the final grade. The third grade is issued after the final assessment at the end of the 

## Example of outputs

In [16]:
multi_query_results

{'question': "I have a question regarding the skill class grade. The skill class grade only affects the final grade for the project correct? Also, from what I understand from the grading and assessment, it's worth a total of 10% of the final grade so if you missed one skill class, you would only miss out on 5% of the final grade, correct?",
 'text': ['1. How does the skill class grade impact the final project grade? Does missing one skill class result in a 5% deduction from the final grade?',
  '2. Can you clarify the relationship between the skill class grade and the final project grade? Is it true that missing one skill class only results in a 5% deduction from the final grade?',
  '3. Regarding the skill class grade, does missing one class affect the final grade by 5%? How does the skill class grade contribute to the overall assessment of the project?']}

In [9]:
retrieve_results # reranker_results and llm_reranker_results are similar

[Document(page_content='Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.', metadata={'article': '5.4', 'article_name': 'attendance and participation', 'document_name': 'rules and regulation', 'page': '21', 'section': '5', 'section_name': 'semester project regulations'}),
 Document(page_content='The project grade is a group grade, which applies to all members of the group. The project examiners may deviate (positively or negatively) from the group grade and issue an individual grade for students, if participation and cooperation within a group has not been homogeneous.', metadata={'article': '5.6', 'article_name': 'project grade', 'document_na

## Pipeline Examples

In [5]:
tmp = "I wanted to ask you a question related to the project meetings and the pre-examination. Two days ago, I had a surgery on my toe in my home country and even though I got discharged, it is recommended to me to visit the hospital every day for monitoring and dressings for the next one or two weeks(depending on my physical condition). My question is, if I show proof about the surgery and the recovery, can I participate in the project meetings and in the pre-examination online instead of on site?"
result = agent.generate_response(tmp, multi_query=True, retrieve_type="vector_search", rerank_type=None)

D:\Code\uni_llm\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




Retriever Results:
1. The Board of Examiners can excuse students from the participation in project meetings in individual cases due to personal circumstances and can offer repair opportunities for the project as a whole. The student must in such cases prove force majeure for all mandatory meeting that the student missed, see also Article 3.7 of the Rules and Regulations. Deviation from the group grade is possible in such cases. The Board of Examiners can also grant resits for projects in special circumstances. Since the project is an onsite group activity, students must be within commuting distance from Maastricht during each project activity.
2. All project meetings are mandatory. One project meeting may be missed each block of the three blocks of a semester, without consequences, as an arrangement to cover force majeure and may only be used as such. If two or more meetings are missed in the first or second block of the semester, the student will not have access to the exam of that 

In [6]:
result.keys()

dict_keys(['context', 'query', 'response'])

In [8]:
print(result["context"])

[Document(page_content='The Board of Examiners can excuse students from the participation in project meetings in individual cases due to personal circumstances and can offer repair opportunities for the project as a whole. The student must in such cases prove force majeure for all mandatory meeting that the student missed, see also Article 3.7 of the Rules and Regulations. Deviation from the group grade is possible in such cases. The Board of Examiners can also grant resits for projects in special circumstances. Since the project is an onsite group activity, students must be within commuting distance from Maastricht during each project activity.', metadata={'article': '5.9', 'article_name': 'project hardship', 'document_name': 'rules and regulation', 'page': '23', 'section': '5', 'section_name': 'semester project regulations'}), Document(page_content='On-campus attendance and participation in the project meetings and project skills trainings, is mandatory, see Article 4.4 of the EER. M

In [6]:
print(result["response"])

Based on the Rules and Regulations, if you provide proof of force majeure, such as your recent surgery and the need for daily monitoring and dressings, the Board of Examiners may excuse you from participating in the project meetings in individual cases. They can offer repair opportunities for the project as a whole. However, since project meetings are mandatory on-site sessions, you may not be able to fully participate online and receive attendance credit. It is recommended to communicate your situation with the Board of Examiners and seek their guidance on how to proceed with your participation in the project meetings and pre-examination given your circumstances.

References: 
- Rules and Regulations, Article: Project Hardship, Page: 23
- Rules and Regulations, Article: Attendance and Participation, Page: 21
- Education and Examination Regulation, Article: Attendance, Participation, and Best-Efforts Obligation, Page: 11

Thanks for asking!


In [12]:
agent.generate_response(query, multi_query=False, retrieve_type="reranker", rerank_type=None)



Reranker Results:
1. The project grade is on a scale from 0 to 10, where the grade consists for 90% of the grade from paragraph 1, to which the skill class grade is added.
2. Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.
3. The project is graded on three occasions by the examiners. The first grade is issued after the presentation first phase and accounts for 15% of the final grade. The second grade is issued after the presentation second phase and accounts for 15% of the final grade. The third grade is issued after the final assessment at the end of the third block and accounts for 70% of the grade. These grades are then rescaled in Para

[Document(page_content='The project is graded on three occasions by the examiners. The first grade is issued after the presentation first phase and accounts for 15% of the final grade. The second grade is issued after the presentation second phase and accounts for 15% of the final grade. The third grade is issued after the final assessment at the end of the third block and accounts for 70% of the grade. These grades are then rescaled in Paragraph 5 to a range 0 to 9.', metadata={'page': '21 & 22', 'article': '5.6', 'article_name': 'project grade', 'document_name': 'rules and regulation', 'section': '5', 'section_name': 'semester project regulations'}),
 Document(page_content='Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the fi

In [13]:
agent.generate_response(query, multi_query=True, retrieve_type="llm_reranker")

  0%|          | 0/77 [00:00<?, ?it/s]

D:\Code\uni_llm\venv\lib\site-packages\transformers\tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(




LLM Reranker Results:
1. The project grade is on a scale from 0 to 10, where the grade consists for 90% of the grade from paragraph 1, to which the skill class grade is added.
2. Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.
3. If a student has participated fully in all but one skill trainings, where fully participating includes that the student has a passing grade for all assignments, the student will receive a grade of 0.5 for the skill classes.
4. The project is graded on three occasions by the examiners. The first grade is issued after the presentation first phase and accounts for 15% of the final grade. The second grade is issued af

  0%|          | 0/77 [00:00<?, ?it/s]



LLM Reranker Results:
1. The project grade is on a scale from 0 to 10, where the grade consists for 90% of the grade from paragraph 1, to which the skill class grade is added.
2. Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.
3. If a student has participated fully in all skill classes, where the student also has a passing grade for all assignments, the student will receive the grade of 1 for the skill trainings.
4. If a student has participated fully in all but one skill trainings, where fully participating includes that the student has a passing grade for all assignments, the student will receive a grade of 0.5 for the skill classes.


  0%|          | 0/77 [00:00<?, ?it/s]



LLM Reranker Results:
1. If a student has participated fully in all but one skill trainings, where fully participating includes that the student has a passing grade for all assignments, the student will receive a grade of 0.5 for the skill classes.
2. Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to participate in the final examination of the project, meaning that an NG is given for the project. This NG will also be awarded if after the final examination it turns out that a student that was ineligible still participated.
3. The project grade is on a scale from 0 to 10, where the grade consists for 90% of the grade from paragraph 1, to which the skill class grade is added.
4. If a student has participated fully in all skill classes, where the student also has a passing grade for all assignments, the student will receive the grade of 1 for the skill trainings.


  0%|          | 0/77 [00:00<?, ?it/s]



LLM Reranker Results:
1. The project grade is on a scale from 0 to 10, where the grade consists for 90% of the grade from paragraph 1, to which the skill class grade is added.
2. If a student has participated fully in all but one skill trainings, where fully participating includes that the student has a passing grade for all assignments, the student will receive a grade of 0.5 for the skill classes.
3. If a student has participated fully in all skill classes, where the student also has a passing grade for all assignments, the student will receive the grade of 1 for the skill trainings.
4. The project is graded on three occasions by the examiners. The first grade is issued after the presentation first phase and accounts for 15% of the final grade. The second grade is issued after the presentation second phase and accounts for 15% of the final grade. The third grade is issued after the final assessment at the end of the third block and accounts for 70% of the grade. These grades are th

[Document(page_content='The project grade is on a scale from 0 to 10, where the grade consists for 90% of the grade from paragraph 1, to which the skill class grade is added.', metadata={'page': '22', 'article': '5.6', 'article_name': 'project grade', 'document_name': 'rules and regulation', 'section': '5', 'section_name': 'semester project regulations'}),
 Document(page_content='If a student has participated fully in all but one skill trainings, where fully participating includes that the student has a passing grade for all assignments, the student will receive a grade of 0.5 for the skill classes.', metadata={'page': '22', 'article': '5.6', 'article_name': 'skill class grade', 'document_name': 'rules and regulation', 'section': '5', 'section_name': 'semester project regulations'}),
 Document(page_content='Participation in project skill trainings is taken into account in the final grading. If a student has missed three or more skills trainings, the student will not be able to particip